In [45]:
import numpy as np
import pandas
import sklearn
from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters, RegexHandler,
                          ConversationHandler)

from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib

In [2]:
token_telegram_bot = '622477280:AAGQnG6ys2bqMq3rkjiHnth5MNcOGVVT9EU'

In [3]:
def retrieve_bot():
    return telegram.Bot(token=token_telegram_bot)

## Chatterbot

In [12]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot

# Create a new chat bot named Charlie
chatbot = ChatBot(
    'teste',
    trainer='chatterbot.trainers.ListTrainer',
    logic_adapters=[
        "chatterbot.logic.MathematicalEvaluation",
        "chatterbot.logic.BestMatch"
    ]
)
chatbot.train([
    "Great!",
    "What are you doing",
    "Your flight has been booked."
])

chatbot.train([
    "Where are you?",
    "I'm here",
    "Great."
])

List Trainer: [####################] 100%
List Trainer: [####################] 100%


In [13]:
# Get a response to the input text 'How are you?'
response = chatbot.get_response("Where are you")

print(response)

I'm here


## Telegram  Bot

In [24]:
from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters, RegexHandler,
                          ConversationHandler)
import logging
from datetime import datetime
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Simple Bot to reply to Telegram messages
# This program is dedicated to the public domain under the CC0 license.

token_telegram_bot = '622477280:AAGQnG6ys2bqMq3rkjiHnth5MNcOGVVT9EU'
"""
This Bot uses the Updater class to handle the bot.
First, a few callback functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.
Usage:
Example of a bot-user conversation using ConversationHandler.
Send /start to initiate the conversation.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)

reply_keyboard = [['Credito', 'Marketing'],
                  ['Como começar?', 'Saude dos negocios'],
                  ['Batata', 'Estudar', 'Done']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)

def start(bot, update, user_data):

    user = update.effective_user
    
    user_data['first_name'] = user.first_name
    user_data['last_name'] = user.last_name
    user_data['telegram_user_id']  = user.id
    
    logging.info(update)
    
    update.message.reply_text(
        "Olá empreendedor, Eu sou a Constelação, sua assistente de negócios",
        reply_markup=markup)
        
    return CHOOSING

def regular_choice(bot, update, user_data):
    text = update.message.text
    user_data['choice'] = text
    update.message.reply_text(
        'Your {}? Yes, I would love to hear about that!'.format(text.lower()))

    return TYPING_REPLY


def custom_choice(bot, update):
    update.message.reply_text('Alright, please send me the category first, '
                              'for example "Most impressive skill"')

    return TYPING_CHOICE


def received_information(bot, update, user_data):
    text = update.message.text
    category = user_data['choice']
    user_data[category] = text
    del user_data['choice']

    update.message.reply_text("Neat! Just so you know, this is what you already told me")

    return CHOOSING


def done(bot, update, user_data):
    
    user_data['_id'] = hash(str(user_data['telegram_user_id']) + str(datetime.now()) + str(update.update_id))
    
    es = Elasticsearch(ELASTICSEARCH_SERVER_ADDRESS)
    es.index(index=DATE_INDEX, doc_type='doc', body=user_data)
    
    if 'choice' in user_data:
        del user_data['choice']

    update.message.reply_text("I learned these facts about you:"
                              "{}"
                              "Until next time!".format(facts_to_str(user_data)))

    user_data.clear()
    return ConversationHandler.END


def error(bot, update, error):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, error)


def telegram_bot():
    # Create the Updater and pass it your bot's token.
    updater = Updater(token_telegram_bot)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher
    
    # Add conversation handler with the states GENDER, PHOTO, LOCATION and BIO
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start, pass_user_data=True)],

        states={
            CHOOSING: [RegexHandler('^(Credito|Marketing|Como começar?|Saude dos negócios)$',
                                    regular_choice,
                                    pass_user_data=True),
                       RegexHandler('^(Batata|Estudar)$',
                                    custom_choice),
                       ],

            TYPING_CHOICE: [MessageHandler(Filters.text,
                                           regular_choice,
                                           pass_user_data=True),
                            ],

            TYPING_REPLY: [MessageHandler(Filters.text,
                                          received_information,
                                          pass_user_data=True),
                           ],
        },

        fallbacks=[RegexHandler('^Done$', done, pass_user_data=True)]
    )

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


In [ ]:
def main():
#     joblib.dump(clf, 'filename.pkl') 
    clf = joblib.load('classifier_kmeans.pkl')

In [26]:
if __name__ == '__main__':
    main()

2018-08-26 00:50:18,320 - root - INFO - {'update_id': 494609538, 'message': {'message_id': 70, 'date': 1535255416, 'chat': {'id': 470319902, 'type': 'private', 'username': 'Falkets', 'first_name': 'Luciano', 'last_name': 'Falqueto'}, 'text': '/start', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 6}], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 470319902, 'first_name': 'Luciano', 'is_bot': False, 'last_name': 'Falqueto', 'username': 'Falkets', 'language_code': 'en-US'}}, '_effective_user': {'id': 470319902, 'first_name': 'Luciano', 'is_bot': False, 'last_name': 'Falqueto', 'username': 'Falkets', 'language_code': 'en-US'}, '_effective_chat': {'id': 470319902, 'type': 'private', 'username': 'Falkets', 'first_name': 'Luciano', 'last_name': 'Falqueto'}}
2018-08-26 00:50:31,256 - root - INFO - {'update_i

## Elasticsearch

In [39]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch.serializer import JSONSerializer
from elasticsearch.helpers import bulk

ELASTICSEARCH_SERVER_ADDRESS = 'localhost'
INDEX_USER_DATA = 'user_data'

# BODY_GET_RAW_DATA = {  "size": 1000,
#                      "query": {
#                          "bool": {
#                              "must": [{"match": {"telegram_user_id": "false"}}]
#                                  }
#                              }
#                         }

BODY_GET_RAW_DATA_BY_USER_ID = {
                                 "size": 1000,
                                 "query": {
                                     "match":{
                                         "telegram_user_id":{}
                                         }
                                    }
                               }

BODY_GET_ALL_DATA = {
                        "query": {
                            "match_all": {}
                        }
                    }

es = Elasticsearch(ELASTICSEARCH_SERVER_ADDRESS)
# insert data in  elastic search
# es.index(index=DATE_INDEX, doc_type='doc', body=USER_DATA)

In [38]:
def get_user_data_query(body, index):
    es = Elasticsearch(ELASTICSEARCH_SERVER_ADDRESS)
    search_val = es.search(index=index, body=body, scroll='1m')
    final_list = []
    
    while search_val['hits']['hits']:
        final_list += [x["_source"] for x in search_val["hits"]["hits"]]
        search_val = es.scroll(scroll_id=search_val['_scroll_id'], scroll='30s')
    df = pd.DataFrame(final_list)
    
    return df

## Clustering

In [44]:
def train_model(model_filename):
    # body = BODY_GET_RAW_DATA_BY_USER_ID
    # body["query"]["telegram_user_id"] = userid
    n_clusters = 5
    df_data = get_user_data_query(BODY_GET_ALL_DATA, INDEX_USER_DATA)

    # X_train, X_test, y_train, y_test = train_test_split(new_data, data[feature_dropped],
    #                                                     train_size=0.25, random_state=42)

    # Kmeans
    clusterer = KMeans(n_clusters=n_clusters).fit(df_data)
    
    # GMM
#     clusterer = GaussianMixture(n_components=n_clusters).fit(reduced_data)
#     preds = clusterer.predict(reduced_data)

    joblib.dump(clusterer, model_filename+'pkl') 
    return clusterer